In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.optimize import minimize

In [3]:

# Liste de symboles de stocks aléatoires
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Télécharger les données des stocks à l'aide de yfinance
data = yf.download(stocks, start="2020-01-01", end="2021-01-01")

# Calcul des rendements quotidiens
returns = data['Adj Close'].pct_change().dropna()

# Calcul de statistiques
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Test de normalité (test de Shapiro-Wilk)
for stock in stocks:
    stat, p = shapiro(returns[stock])
    print(f"Test de normalité pour {stock}: Statistique={stat}, p-value={p}")
  

[*********************100%%**********************]  5 of 5 completed
Test de normalité pour AAPL: Statistique=0.9396017789840698, p-value=1.1503578534188819e-08
Test de normalité pour MSFT: Statistique=0.911160409450531, p-value=4.41174770327013e-11
Test de normalité pour GOOGL: Statistique=0.9328373670578003, p-value=2.692721601249559e-09
Test de normalité pour AMZN: Statistique=0.9804868698120117, p-value=0.0015486348420381546
Test de normalité pour TSLA: Statistique=0.9601470232009888, p-value=1.918489715535543e-06


In [5]:
  
# Détermination de la variance minimale
def minimize_variance(weights):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Contraintes d'allocation
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Bornes des poids
bounds = tuple((0, 1) for stock in range(len(stocks)))

# Initialisation des poids égaux
initial_weights = len(stocks) * [1. / len(stocks)]

# Optimisation pour la variance minimale
min_var_portfolio = minimize(minimize_variance, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)


In [7]:

print("Poids du portefeuille à variance minimale:")
for i in range(len(stocks)):
    print(f"{stocks[i]}: {min_var_portfolio.x[i]}")

# Création de la frontière efficiente
from scipy.optimize import minimize

def efficient_frontier(returns):
    num_assets = len(returns.columns)
    results = np.zeros((4, num_assets))
    
    for i in range(num_assets):
        weights = np.array([1 / num_assets] * num_assets)
        
        def portfolio_variance(weights):
            return np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
        
        constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
        
        bounds = tuple((0, 1) for asset in range(num_assets))
        
        result = minimize(portfolio_variance, weights, method='SLSQP', bounds=bounds, constraints=constraints)
        
        portfolio_stddev = result.fun
        portfolio_return = np.sum(weights * returns.mean()) * 252
        
        results[0, i] = portfolio_stddev
        results[1, i] = portfolio_return
        results[2, i] = portfolio_return / portfolio_stddev
        results[3, i] = weights
        
    return results


Poids du portefeuille à variance minimale:
AAPL: 0.0
MSFT: 0.5432539410112296
GOOGL: 0.4567460589887703
AMZN: 0.0
TSLA: 0.0


In [16]:
def efficient_frontier(returns):
    num_assets = len(returns.columns)
    results = np.zeros((4, num_assets), dtype=object)  # Use dtype=object for non-homogeneous data
    
    for i in range(num_assets):
        weights = np.array([1 / num_assets] * num_assets)  # Convert to NumPy array
        
        def portfolio_variance(weights):
            return np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
        
        constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
        
        bounds = tuple((0, 1) for asset in range(num_assets))
        
        result = minimize(portfolio_variance, weights, method='SLSQP', bounds=bounds, constraints=constraints)
        
        portfolio_stddev = result.fun
        portfolio_return = np.sum(weights * returns.mean()) * 252
        
        results[0, i] = portfolio_stddev
        results[1, i] = portfolio_return
        results[2, i] = portfolio_return / portfolio_stddev
        results[3, i] = result.x  # Assign the optimized weights as a NumPy array
        
    return results

efficient_frontier_data = efficient_frontier(returns)


In [18]:

efficient_frontier_df = pd.DataFrame(efficient_frontier_data.T, columns=["Risk", "Return", "Sharpe Ratio", "Weights"])

In [19]:

print("Frontière efficiente:")
print(efficient_frontier_df)



Frontière efficiente:
       Risk    Return Sharpe Ratio  \
0  0.022208  0.913813    41.147686   
1  0.022208  0.913813    41.147686   
2  0.022208  0.913813    41.147686   
3  0.022208  0.913813    41.147686   
4  0.022208  0.913813    41.147686   

                                             Weights  
0  [0.0, 0.5432539410112296, 0.4567460589887703, ...  
1  [0.0, 0.5432539410112296, 0.4567460589887703, ...  
2  [0.0, 0.5432539410112296, 0.4567460589887703, ...  
3  [0.0, 0.5432539410112296, 0.4567460589887703, ...  
4  [0.0, 0.5432539410112296, 0.4567460589887703, ...  
